# API Face Samples

## Objective

This script manages customer checkin data within a system by utilizing Azure Dynamic Person Group. It automates the detection and enrollment of customer faces, identifies customers within dynamic person groups (such as daily and location-specific groups), and links customers to these groups to streamline face recognition processes.

In [ ]:
import os
import logging
import sys
from shared_functions import (
    add_person_face, 
    create_person,
    create_dynamic_person_group,
    check_dynamic_person_group_exists,
    detect_faces,
    delete_dynamic_person_group,
    delete_person,
    identify_faces,
    link_person_to_dynamic_person_group
)
from datetime import datetime

# Replace with your Azure Face API subscription key and endpoint
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]

image_path = "path_to_image.jpg"
patient_name = "Patient Name" # Optional
customer_group_id = "customer_group_id"
daily_group_id = f"daily_group_{customer_group_id}_{datetime.now().strftime('%Y_%m_%d')}"

logging.basicConfig(level=logging.INFO)

def handle_error(message):
    logging.error(message)
    sys.exit(1)

try:
    # Ensure both daily and customer dynamic groups exist
    if not check_dynamic_person_group_exists(FACE_KEY, FACE_ENDPOINT, daily_group_id, "sample=customer_checkin_management"):
        create_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, daily_group_id,  "sample=customer_checkin_management")
        logging.info(f"Created daily group: {daily_group_id}")
    else:
        logging.info(f"Daily group already exists: {daily_group_id}")

    if not check_dynamic_person_group_exists(FACE_KEY, FACE_ENDPOINT, customer_group_id, "sample=customer_checkin_management"):
        create_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, customer_group_id, "sample=customer_checkin_management")
        logging.info(f"Created customer group: {customer_group_id}")
    else:
        logging.info(f"Customer group already exists: {customer_group_id}")
except Exception as e:
    handle_error(f"Error checking or creating dynamic groups: {str(e)}")

try:
    # Detect faces in the image
    detected_faces = detect_faces(FACE_KEY, FACE_ENDPOINT, image_path, "sample=customer_checkin_management")
    if len(detected_faces) == 0:
        logging.info("No faces detected in the image.")
        sys.exit(0)
except Exception as e:
    handle_error(f"Error detecting faces: {str(e)}")

face_id = detected_faces[0]["faceId"]

try:
    # Check if the face exists in the daily group
    person_id = identify_faces(FACE_KEY, FACE_ENDPOINT, face_id, daily_group_id, "sample=customer_checkin_management")

    if not person_id:
        logging.info("Person not found in daily group.")
        # Check if the face exists in the customer group
        person_id = identify_faces(FACE_KEY, FACE_ENDPOINT, face_id, customer_group_id, "sample=customer_checkin_management")

        if person_id:
            logging.info("Person exists in customer group.")
            if link_person_to_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, daily_group_id, person_id, "sample=customer_checkin_management"):
                logging.info("Person linked to daily group.")
            else:
                logging.warning("Failed to link person to daily group.")
        else:
            # If person not found, create a new person and add their face
            logging.info("Person does not exist in daily or customer group. Creating new person.")
            person_id = create_person(FACE_KEY, FACE_ENDPOINT, patient_name, "sample=customer_checkin_management")
            if not person_id:
                handle_error("Failed to create person.")
            persisted_face_id = add_person_face(FACE_KEY, FACE_ENDPOINT, image_path, person_id, "sample=customer_checkin_management")
            if not persisted_face_id:
                handle_error("Failed to add face to person.")

            # Link the person to the daily and customer group
            if link_person_to_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, daily_group_id, person_id, "sample=customer_checkin_management"):
                logging.info("Person linked to daily group.")
            else:
                logging.warning("Failed to link person to daily group.")
            if link_person_to_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, customer_group_id, person_id, "sample=customer_checkin_management"):
                logging.info("Person linked to customer group.")
            else:
                logging.warning("Failed to link person to customer group.")
            logging.info("Person linked to both daily and customer groups.")
    else:
        logging.info("Person found in daily group.")
except Exception as e:
    handle_error(f"Error identifying or linking person: {str(e)}")

# Optionally delete dynamic person groups and the person 
try:
    if delete_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, daily_group_id, "sample=customer_checkin_management"):
        logging.info(f"Deleted daily group: {daily_group_id}")
    else:
        logging.warning(f"Failed to delete daily group: {daily_group_id}")

    if delete_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, customer_group_id, "sample=customer_checkin_management"):
        logging.info(f"Deleted customer group: {customer_group_id}")
    else:
        logging.warning(f"Failed to delete customer group: {customer_group_id}")
    if delete_person(FACE_KEY, FACE_ENDPOINT, person_id, "sample=customer_checkin_management"):
        logging.info(f"Deleted person: {person_id}")
    else:
        logging.warning(f"Failed to delete person: {person_id}")
except Exception as e:
    handle_error(f"Error deleting person or daily/customer groups: {str(e)}")

logging.info("Process completed successfully.")